# RNN Tensorflow

## Character Sequence RNN
Long sentence

In [1]:
import tensorflow as tf
import numpy as np
from time import time
tf.set_random_seed(777)  # reproducibility

In [2]:
sample = ("if you want to build a ship, don't drum up people together to"
            "collect wood and don't assign them tasks and work, but rather"
            "teach them to long for the endless immensity of the sea.")

idx2char = list(set(sample))  # 고유한 char 만 추출
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex
print(' idx2char :', idx2char, '\n\n', 'char2idx :', char2idx)

 idx2char : ['d', 's', 'r', 'w', 'k', 'm', 'e', 'p', "'", 'b', ',', 'f', 'o', 'g', 'h', 'c', 'y', '.', ' ', 'u', 'l', 't', 'i', 'a', 'n'] 

 char2idx : {'d': 0, 's': 1, 'r': 2, 'w': 3, 'k': 4, 'm': 5, 'e': 6, 'p': 7, "'": 8, 'b': 9, ',': 10, 'f': 11, 'o': 12, 'g': 13, 'h': 14, 'c': 15, 'y': 16, '.': 17, ' ': 18, 'u': 19, 'l': 20, 't': 21, 'i': 22, 'a': 23, 'n': 24}


In [3]:
# hyper parameters
dic_size = len(char2idx)     # 입력값             : RNN input size (one hot size)
hidden_size = len(char2idx)  # 각 cell의 Hidden값 : RNN output size
num_classes = len(char2idx)  # 출력값             : final output size (RNN or softmax, etc.)
batch_size = 1                     # 1번 처리시 1줄씩 입력 : one sample data, one batch
sequence_length = len(sample) - 1  # 1번 처리시 활용할 Cell(단어)의 수 : number of lstm rollings (unit #)
learning_rate = 0.1

In [4]:
# 데이터 구조형식을 정의
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [5]:
# # RNN Cell 들을 정의
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

In [6]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

In [7]:
# Testing a GPU graph
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [8]:
# Creates a graph. ( by GPU )
from time import time
t0 = time()
with tf.device('/gpu:0'):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(203):
            l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
            result = sess.run(prediction, feed_dict={X: x_data})
            # print char using dic
            result_str = [idx2char[c] for c in np.squeeze(result)]
            if i % 25 == 0:
                print(i, "loss:", l, "Prediction:", ''.join(result_str))
            if i == 202:
                texts = ''.join(result_str)
print(  int(time()-t0) , 'sec')

0 loss: 3.22625 Prediction:                                                                                                                                                                                  
25 loss: 0.307909 Prediction: f you want to build a ship, don't wrum up people together tocollect wool and don't wssign them tasks and work, bui ratherteach them to long for the endless immensity of the sea.
50 loss: 0.010529 Prediction: f you want to build a ship, don't drum up people together tocollect wood and don't assign them tasks and work, but ratherteach them to long for the endless immensity of the sea.
75 loss: 0.00200075 Prediction: f you want to build a ship, don't drum up people together tocollect wood and don't assign them tasks and work, but ratherteach them to long for the endless immensity of the sea.
100 loss: 0.00118363 Prediction: f you want to build a ship, don't drum up people together tocollect wood and don't assign them tasks and work, but ratherteach them to 

In [9]:
# # Creates a graph. ( by CPU )
# t1 = time()
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(50):
#         l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
#         result = sess.run(prediction, feed_dict={X: x_data})
#         # print char using dic
#         result_str = [idx2char[c] for c in np.squeeze(result)]
#         if i % 10 == 0:
#             print(i, "loss:", l, "Prediction:", ''.join(result_str))
# print( int(time()-t1), 'sec')

## Character Sequence RNN 2
Long sentence by MultiRNNCell (cell을 몇 층으로 쌓을 것인가?)

    각 sequence 의 'hidden' 결과를 모두 모은 뒤, softmax로 가장 적합한 조합문장을 찾는다. 

In [10]:
from __future__ import print_function
# import tensorflow as tf
# import numpy as np
from tensorflow.contrib import rnn
# tf.set_random_seed(777)  # reproducibility
t2 = time()

In [11]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

In [12]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

In [13]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)
    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index
    dataX.append(x)
    dataY.append(y)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [14]:
batch_size = len(dataX)
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])
# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape

Tensor("one_hot_1:0", shape=(?, 10, 25), dtype=float32)


In [15]:
# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

In [16]:
multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)
# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [17]:
# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [18]:
t1 = time()
tf.device('/gpu:0')
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        if i % 50 == 0: print(i, j, ''.join([char_set[t] for t in index]), l)

0 0 .lffffffff 3.22003
0 1 ,fffffffff 3.22003
0 2 ffffffffff 3.22003
0 3 .kkfffffff 3.22003
0 4 n,ffffffff 3.22003
0 5 ,fffffffff 3.22003
0 6 ffffcfffff 3.22003
0 7 rccccccfff 3.22003
0 8 ccccccffff 3.22003
0 9 r..fffffff 3.22003
0 10 kffkffffff 3.22003
0 11 ffffffffff 3.22003
0 12 kkkfffffff 3.22003
0 13 nfffffffff 3.22003
0 14 fffffhhhff 3.22003
0 15 ttthhhhhff 3.22003
0 16 ,mmhhhmmmm 3.22003
0 17 .mhhrrmfmm 3.22003
0 18 hhhhfffmmm 3.22003
0 19 ggrmmfmmmm 3.22003
0 20 ffffffmmmm 3.22003
0 21 cfff.mmmmm 3.22003
0 22 fff.mmmmmr 3.22003
0 23 tb..wmmrrc 3.22003
0 24 rrmmmmrrcc 3.22003
0 25 .mmmmmrccc 3.22003
0 26 aammgrcccc 3.22003
0 27 ofoorccccr 3.22003
0 28 ffgrccccrr 3.22003
0 29 ggrccccrrr 3.22003
0 30 nccccccrkm 3.22003
0 31 rcc..rrkmm 3.22003
0 32 ckffkkkkmm 3.22003
0 33 kfsspkppmm 3.22003
0 34 ffgrmmmmmm 3.22003
0 35 ggrkkmmmmm 3.22003
0 36 tkkkmmmmmm 3.22003
0 37 ,l,,,mmmma 3.22003
0 38  m,mmmmmma 3.22003
0 39 ffffmmmmam 3.22003
0 40 ,ammmmmamm 3.22003
0 41 ammmmmmmmf 3.22003
0 

100 0 m you want 0.235059
100 1 oyou want  0.235059
100 2 tou want t 0.235059
100 3 ou want to 0.235059
100 4   want to  0.235059
100 5  want to b 0.235059
100 6 tont to bu 0.235059
100 7 ont to bui 0.235059
100 8 nd to buil 0.235059
100 9 d to build 0.235059
100 10  do build  0.235059
100 11 to build a 0.235059
100 12   cuild a  0.235059
100 13  build a s 0.235059
100 14 tuild a sh 0.235059
100 15 utld a shi 0.235059
100 16  ld a ship 0.235059
100 17 md a ship, 0.235059
100 18 e a ship,  0.235059
100 19  a ship, d 0.235059
100 20 tnship, do 0.235059
100 21 nship, don 0.235059
100 22 thip, don' 0.235059
100 23 sip, don't 0.235059
100 24 ep, don't  0.235059
100 25 m, don't d 0.235059
100 26   don't dr 0.235059
100 27  don't dru 0.235059
100 28 ton't arum 0.235059
100 29  n't drum  0.235059
100 30  't arum u 0.235059
100 31 dt drum up 0.235059
100 32 t drum up  0.235059
100 33  drum up p 0.235059
100 34 toum up pe 0.235059
100 35  um up peo 0.235059
100 36  m up peop 0.235059
100 37   up

200 0 f you want 0.230381
200 1  you want  0.230381
200 2 tou want t 0.230381
200 3  u want to 0.230381
200 4 n want to  0.230381
200 5 pwant to b 0.230381
200 6 tont to bu 0.230381
200 7 ont to bui 0.230381
200 8 nd to buil 0.230381
200 9 d to build 0.230381
200 10 hao build  0.230381
200 11 to luild a 0.230381
200 12 h luild a  0.230381
200 13 nluild a s 0.230381
200 14 tutld a sh 0.230381
200 15 utld a shi 0.230381
200 16 pld a ship 0.230381
200 17 fd a ship, 0.230381
200 18 e a ship,  0.230381
200 19  a ship, d 0.230381
200 20 tnship, do 0.230381
200 21 nship, don 0.230381
200 22 thip, don' 0.230381
200 23  ip, don't 0.230381
200 24 ep, don't  0.230381
200 25 f, don't d 0.230381
200 26 , don't dr 0.230381
200 27  bon't dru 0.230381
200 28 ton't arum 0.230381
200 29  n't arum  0.230381
200 30 n't drum u 0.230381
200 31 dt arum up 0.230381
200 32 t arum up  0.230381
200 33 harum up p 0.230381
200 34 toum up pe 0.230381
200 35  um up peo 0.230381
200 36  m up peop 0.230381
200 37 p up

300 0 l you want 0.229767
300 1  you want  0.229767
300 2 tou want t 0.229767
300 3  u want to 0.229767
300 4   want to  0.229767
300 5 iwant to b 0.229767
300 6 tont to bu 0.229767
300 7 ont to bui 0.229767
300 8 nd to buil 0.229767
300 9 d to build 0.229767
300 10  ao build  0.229767
300 11 to cuild a 0.229767
300 12   cuild a  0.229767
300 13  luild a s 0.229767
300 14 tuild a sh 0.229767
300 15 uild a shi 0.229767
300 16 ild a ship 0.229767
300 17 ld a ship, 0.229767
300 18 e a ship,  0.229767
300 19  a ship, d 0.229767
300 20 tnship, do 0.229767
300 21 nship, don 0.229767
300 22 thip, don' 0.229767
300 23 sip, don't 0.229767
300 24 ep, don't  0.229767
300 25 l, don't d 0.229767
300 26   don't dr 0.229767
300 27  don't dru 0.229767
300 28 ton't drum 0.229767
300 29  n't drum  0.229767
300 30  't arum u 0.229767
300 31 dt drum up 0.229767
300 32 t drum up  0.229767
300 33  arum up p 0.229767
300 34 toum up pe 0.229767
300 35  um up peo 0.229767
300 36  m up peop 0.229767
300 37 i up

400 0 t you want 0.229046
400 1  you want  0.229046
400 2 tou want t 0.229046
400 3  u want to 0.229046
400 4   want to  0.229046
400 5  want to b 0.229046
400 6 tont to bu 0.229046
400 7 ont to bui 0.229046
400 8 nd to buil 0.229046
400 9 d to build 0.229046
400 10  to build  0.229046
400 11 to luild a 0.229046
400 12   luild a  0.229046
400 13  build a s 0.229046
400 14 tuild a sh 0.229046
400 15 uild a shi 0.229046
400 16  ld a ship 0.229046
400 17 td a ship, 0.229046
400 18 e a ship,  0.229046
400 19  a ship, d 0.229046
400 20 tnship, do 0.229046
400 21 nship, don 0.229046
400 22 thip, don' 0.229046
400 23  ip, don't 0.229046
400 24 ep, don't  0.229046
400 25 t, don't d 0.229046
400 26   don't dr 0.229046
400 27  bon't dru 0.229046
400 28 ton't drum 0.229046
400 29  n't drum  0.229046
400 30  't drum u 0.229046
400 31 dt drum up 0.229046
400 32 t drum up  0.229046
400 33  trum up p 0.229046
400 34 toum up pe 0.229046
400 35  um up peo 0.229046
400 36  m up peop 0.229046
400 37   up

In [19]:
# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')
print( int(time()-t2), 'sec')

p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.6 sec


In [20]:
# Compare with first method
texts

"f you want to build a ship, don't drum up people together tocollect wood and don't assign them tasks and work, but ratherteach them to long for the endless immensity of the sea."